In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K


from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten
from keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.utils import array_to_img, img_to_array, load_img
import matplotlib.image as mpimg
tf.print('Using TensorFlow version: ' + tf.__version__)
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


2023-01-01 19:31:23.361360: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-01 19:31:23.530426: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-01-01 19:31:23.533526: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-01-01 19:31:23.533548: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore 

In [2]:
matplotlib.use('Agg')
tf.random.set_seed(42)
np.random.seed(42)

In [3]:

def load_images(folder,lettre):
    images = []
   

    for filename in os.listdir(folder+"/"+lettre):
        img = mpimg.imread(os.path.join(folder+"/"+lettre, filename))
        if img is not None:
            images.append(img)
            
    return np.asarray(images)

In [4]:
Datafolder=["A","B","C","D","E","F","G","H","I","J"]
X_train=[]




In [5]:
for label in Datafolder:
    Xt=load_images("data",label)
    X_train.append(Xt)
 
  


In [6]:

X_train_dcgan =  [x.reshape(-1, 28, 28, 1) * 2. - 1. for x in X_train] # reshape and rescale

In [7]:
codings_size = 100

generateur

In [8]:
generator=[]
for i in range(10):
    cnn= keras.models.Sequential([
    keras.layers.Dense(7 * 7 * 128, input_shape=[codings_size]),
    keras.layers.Reshape([7, 7, 128]),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2DTranspose(64, kernel_size=5, strides=2, padding="SAME",
    activation="selu"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2DTranspose(1, kernel_size=5, strides=2, padding="SAME",
    activation="tanh"),
    ])
    generator.append(cnn)

2023-01-01 19:31:29.879736: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-01-01 19:31:29.879799: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (diarra-ASUS-TUF-Dash-F15-FX516PR-FX516PR): /proc/driver/nvidia/version does not exist
2023-01-01 19:31:29.880566: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


discriminateur

In [9]:
discriminator =[]

for i in range(10):
    cnn=keras.models.Sequential([
    keras.layers.Conv2D(64, kernel_size=5, strides=2, padding="SAME",
    activation=keras.layers.LeakyReLU(0.2),
    input_shape=[28, 28, 1]),
    keras.layers.Dropout(0.4),
    keras.layers.Conv2D(128, kernel_size=5, strides=2, padding="SAME",
    activation=keras.layers.LeakyReLU(0.2)),
    keras.layers.Dropout(0.4),
    keras.layers.Flatten(),
    keras.layers.Dense(1, activation="sigmoid")])
    discriminator.append(cnn)

In [10]:
gan =[]
for i in range(10):
 g=keras.models.Sequential([generator[i], discriminator[i]])
 gan.append(g)

In [11]:
batch_size = 32
dataset =[tf.data.Dataset.from_tensor_slices(x) for x in X_train_dcgan] 
dataset = [x.shuffle(1000) for x in dataset]
dataset =[x.batch(batch_size, drop_remainder=True).prefetch(1) for x in dataset] 

affichage des images 

In [12]:
def plot_multiple_images(images, n_cols=None):
    n_cols = n_cols or len(images)
    n_rows = (len(images) - 1) // n_cols + 1
    if images.shape[-1] == 1:
        images = np.squeeze(images, axis=-1)
    plt.figure(figsize=(n_cols, n_rows))
    for index, image in enumerate(images):
        plt.subplot(n_rows, n_cols, index + 1)
        plt.imshow(image, cmap="binary")
        plt.axis("off")

enregistrement des images 

In [13]:
def save_fig(lettre,fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join("image/"+lettre, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
     plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

entrainement 

In [14]:
def train_gan(lettre,gan, dataset, batch_size, codings_size, n_epochs=50):
    generator, discriminator = gan.layers
    for epoch in range(n_epochs):
        print("Epoch {}/{}".format(epoch + 1, n_epochs))
        for X_batch in dataset:
            # phase 1 - training the discriminator
            noise = tf.random.normal(shape=[batch_size, codings_size])
            generated_images = generator(noise)
            X_fake_and_real = tf.concat([generated_images, X_batch], axis=0)
            y1 = tf.constant([[0.]] * batch_size + [[1.]] * batch_size)
            discriminator.trainable = True
            discriminator.train_on_batch(X_fake_and_real, y1)
            # phase 2 - training the generator
            noise = tf.random.normal(shape=[batch_size, codings_size])
            y2 = tf.constant([[1.]] * batch_size)
            discriminator.trainable = False
            gan.train_on_batch(noise, y2)
       # plot_multiple_images(generated_images, 8)
       # save_fig(lettre,str(epoch))
    generator.save("generateur"+lettre+".h5")
    discriminator.save("discriminateur"+lettre+".h5")

In [15]:
for i in range(10):

    dis=discriminator[i]
    ga= gan[i]
    with tf.device('/gpu:0'):
        dis.compile(loss="binary_crossentropy", optimizer="rmsprop")
        dis.trainable = False
        ga.compile(loss="binary_crossentropy", optimizer="rmsprop")
        train_gan(Datafolder[i],ga, dataset[i], batch_size, codings_size,50)

Epoch 1/50
Saving figure 0
Epoch 2/50
Saving figure 1
Epoch 3/50
Saving figure 2
Epoch 4/50
Saving figure 3
Epoch 5/50
Saving figure 4
Epoch 6/50
Saving figure 5
Epoch 7/50
Saving figure 6
Epoch 8/50
Saving figure 7
Epoch 9/50
Saving figure 8
Epoch 10/50
Saving figure 9
Epoch 11/50
Saving figure 10
Epoch 12/50
Saving figure 11
Epoch 13/50
Saving figure 12
Epoch 14/50
Saving figure 13
Epoch 15/50
Saving figure 14
Epoch 16/50
Saving figure 15
Epoch 17/50
Saving figure 16
Epoch 18/50
Saving figure 17
Epoch 19/50
Saving figure 18
Epoch 20/50
Saving figure 19
Epoch 21/50


/tmp/ipykernel_1164492/3458115351.py:6: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  plt.figure(figsize=(n_cols, n_rows))


Saving figure 20
Epoch 22/50
Saving figure 21
Epoch 23/50
Saving figure 22
Epoch 24/50
Saving figure 23
Epoch 25/50
Saving figure 24
Epoch 26/50
Saving figure 25
Epoch 27/50
Saving figure 26
Epoch 28/50
Saving figure 27
Epoch 29/50
Saving figure 28
Epoch 30/50
Saving figure 29
Epoch 31/50
Saving figure 30
Epoch 32/50
Saving figure 31
Epoch 33/50
Saving figure 32
Epoch 34/50
Saving figure 33
Epoch 35/50
Saving figure 34
Epoch 36/50
Saving figure 35
Epoch 37/50
Saving figure 36
Epoch 38/50
Saving figure 37
Epoch 39/50
Saving figure 38
Epoch 40/50
Saving figure 39
Epoch 41/50
Saving figure 40
Epoch 42/50
Saving figure 41
Epoch 43/50
Saving figure 42
Epoch 44/50
Saving figure 43
Epoch 45/50
Saving figure 44
Epoch 46/50
Saving figure 45
Epoch 47/50
Saving figure 46
Epoch 48/50
Saving figure 47
Epoch 49/50
Saving figure 48
Epoch 50/50
Saving figure 49
Epoch 1/50
Saving figure 0
Epoch 2/50
Saving figure 1
Epoch 3/50
Saving figure 2
Epoch 4/50
Saving figure 3
Epoch 5/50
Saving figure 4
Epoch 6

In [16]:
def save(nb):
    for j in range(10):
         for i in range(nb):
            noise = tf.random.normal(shape=[1, codings_size])
            generated_images = generator[j](noise,1)
            images = np.squeeze(generated_images, axis=-1)
            for index, image in enumerate(images):
                plt.imsave("images/"+Datafolder[j]+"/"+str(i)+".png",image, cmap="binary")
    


In [ ]:
save(10)

enregistrement des models

In [ ]:
for j in range(10):     
 g= gan[j] 
 generateur, discriminateur = g.layers
 generateur.save("generateur"+Datafolder[j]+".h5")
 discriminateur.save("discriminateur"+Datafolder[j]+".h5")
  

In [19]:
modelA = keras.models.load_model('generateurA.h5')



In [20]:
noise = tf.random.normal(shape=[1, codings_size])
generated_images = modelA(noise,1)
images = np.squeeze(generated_images, axis=-1)
for index, image in enumerate(images):
    plt.imsave("images/test/"+str(i)+".png",image, cmap="binary")
    